# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 57 new papers today
          15 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/14 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.03420


extracting tarball to tmp_2303.03420...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.03420/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:318: LatexWarning: Latex injecting: 'appendix_confidences' from 'tmp_2303.03420/appendix_confidences.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:318: LatexWarning: Latex injecting: 'appendix_queries' from 'tmp_2303.03420/appendix_queries.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/

...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.03824/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2303.03936...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.03952


extracting tarball to tmp_2303.03952...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03424-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03424) | **SN2017egm: A Helium-rich Superluminous Supernova with Multiple Bumps in  the Light Curves**  |
|| Jiazheng Zhu, et al. -- incl., <mark>Peter J. Brown</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *25 pages, 14 Figures, 4 Tables; accepted for publication in ApJ (Mar. 2023)*|
|**Abstract**| When discovered, SN~2017egm was the closest (redshift $z=0.03$) hydrogen-poor superluminous supernova (SLSN-I) and a rare case that exploded in a massive and metal-rich galaxy. Thus, it has since been extensively observed and studied. We report spectroscopic data showing strong emission at around He~I $\lambda$10,830 and four He~I absorption lines in the optical. Consequently, we classify SN~2017egm as a member of an emerging population of helium-rich SLSNe-I (i.e., SLSNe-Ib). We also present our late-time photometric observations. By combining them with archival data, we analyze high-cadence ultra-violet, optical, and near-infrared light curves spanning from early pre-peak ($\sim -20\,d$) to late phases ($\sim +300\,d$). We obtain its most complete bolometric light curve, in which multiple bumps are identified. None of the previously proposed models can satisfactorily explain all main light-curve features, while multiple interactions between the ejecta and circumstellar material (CSM) may explain the undulating features. The prominent infrared excess with a blackbody luminosity of $10^7$--$10^8\,L_{sun}$ detected in SN~2017egm could originate from the emission of either an echo of a pre-existing dust shell, or newly-formed dust, offering an additional piece of evidence supporting the ejecta-CSM interaction model. Moreover, our analysis of deep $Chandra$ observations yields the tightest-ever constraint on the X-ray emission of an SLSN-I, amounting to an X-ray-to-optical luminosity ratio $\lesssim 10^{-3}$ at late phases ($\sim100-200\,d$), which could help explore its close environment and central engine. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03435-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03435) | **Population III X-ray Binaries and their Impact on the Early Universe**  |
|| Nina S. Sartorio, et al. -- incl., <mark>M. Magg</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *Accepted for publication in MNRAS, 17 pages, 9 figures*|
|**Abstract**| The first population of X-ray binaries (XRBs) is expected to affect the thermal and ionization states of the gas in the early Universe. Although these X-ray sources are predicted to have important implications for high-redshift observable signals, such as the hydrogen 21-cm signal from cosmic dawn and the cosmic X-ray background, their properties are poorly explored, leaving theoretical models largely uninformed. In this paper we model a population of X-ray binaries arising from zero metallicity stars. We explore how their properties depend on the adopted initial mass function (IMF) of primordial stars, finding a strong effect on their number and X-ray production efficiency. We also present scaling relations between XRBs and their X-ray emission with the local star formation rate, which can be used in sub-grid models in numerical simulations to improve the X-ray feedback prescriptions. Specifically, we find that the uniformity and strength of the X-ray feedback in the intergalactic medium is strongly dependant on the IMF. Bottom-heavy IMFs result in a smoother distribution of XRBs, but have a luminosity orders of magnitude lower than more top-heavy IMFs. Top-heavy IMFs lead to more spatially uneven, albeit strong, X-ray emission. An intermediate IMF has a strong X-ray feedback while sustaining an even emission across the intergalactic medium. These differences in X-ray feedback could be probed in the future with measurements of the cosmic dawn 21-cm line of neutral hydrogen, which offers us a new way of constraining population III IMF. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03438-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03438) | **X-ray Unveiling Events in a z~1.6 Active Galactic Nucleus in the 7 Ms  Chandra Deep Field-South**  |
|| <mark>Li-Ming Yu</mark>, et al. |
|*Appeared on*| *2023-03-08*|
|*Comments*| *23 pages, 9 figures, accepted for publication in ApJ*|
|**Abstract**| We investigate the extreme X-ray variability of a z = 1.608 active galactic nucleus in the 7 Ms Chandra Deep Field-South (XID 403), which showed two significant X-ray brightening events. In the first event, XID 403 brightened by a factor of $>2.5$ in $\lesssim6.1$ rest-frame days in the observed-frame 0.5-5 keV band. The event lasted for $\approx5.0\textrm{-}7.3$ days, and then XID 403 dimmed by a factor of $>6.0$ in $\lesssim6.1$ days. After $\approx1.1\textrm{-}2.5$ years in the rest frame (including long observational gaps), it brightened again with the 0.5-5 keV flux increasing by a factor of $>12.6$. The second event lasted over 251 days and the source remained bright until the end of the 7 Ms exposure. The spectrum is a steep power law (photon index $\Gamma=2.8\pm0.3$) without obscuration during the second outburst, and the rest-frame 2-10 keV luminosity reaches $1.5^{+0.8}_{-0.5}\times10^{43}$ erg s$^{-1}$; there is no significant spectral evolution within this epoch. The infrared-to-UV spectral energy distribution of XID 403 is dominated by the host galaxy. There is no significant optical/UV variability and $R$-band (rest-frame $\approx2500$ $\unicode{xC5}$) brightening contemporaneous with the X-ray brightening. The extreme X-ray variability is likely due to two X-ray unveiling events, where the line of sight to the corona is no longer shielded by high-density gas clumps in a small-scale dust-free absorber. XID 403 is probably a high-redshift analog of local narrow-line Seyfert 1 galaxies, and the X-ray absorber is a powerful accretion-disk wind. On the other hand, we cannot exclude the possibility that XID 403 is an unusual candidate for tidal disruption events. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03824-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03824) | **A new pulsar timing model for scalar-tensor gravity with applications to  PSR J2222-0137 and pulsar-black hole binaries**  |
|| A. Batrakov, et al. -- incl., <mark>M. Kramer</mark>, <mark>Y. J. Guo</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *Accepted to Astronomy and Astrophysics on 09.02.2023*|
|**Abstract**| Context. Scalar-tensor gravity (STG) theories are well-motivated alternatives to general relativity (GR). One class of STG theories, the Damour-Esposito-Farese (DEF) gravity, has a massless scalar field with two arbitrary coupling parameters. We are interested in this theory because, despite its simplicity, it predicts a wealth of different phenomena, such as dipolar gravitational wave emission and spontaneous scalarization of neutron stars (NSs). These phenomena of DEF gravity can be tested by timing binary radio pulsars. Aims. We aim to develop a new binary pulsar timing model DDSTG to enable more precise tests of STG theories based on a minimal set of binary parameters. The expressions for post-Keplerian (PK) parameters in DEF gravity are self-consistently incorporated into the model. The new technique takes into account all possible correlations between PK parameters naturally. Methods. Grids of physical parameters of NSs are calculated in the framework of DEF gravity for a set of 11 equations of state. The automatic Differentiation (AutoDiff) technique is employed, which aids in the calculation of gravitational form factors of NSs with higher precision than in previous works. The pulsar timing program TEMPO is selected as a framework for the realization of the DDSTG model. The implemented model is applicable to any type of pulsar companions. Results. We apply the DDSTG model to the most recently published observational data for PSR J2222-0137. The obtained limits on DEF gravity parameters for this system confirm and improve previous results. New limits are also the most reliable because DEF gravity is directly fitted to the data. We argue that future observations of PSR J2222-0137 can significantly improve the limits and that PSR-BH systems have the potential to place the tightest limits in certain areas of the DEF gravity parameter space. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03936-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03936) | **Using entropy bounds to avoid the cosmological singularity and constrain  particle production**  |
|| Hao Yu, <mark>Jin Li</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *21 pages, 7 figures*|
|**Abstract**| In this work, we study the applications of entropy bounds in two toy cosmological models with particle production (annihilation), i.e., radiation-dominated universe and dust-dominated universe. Since entropy bounds are involved in the volume of the thermodynamc system, we need to specify the thermodynamc system in the universe in advance. We consider the co-moving volume and the volume covered by the particle horizon as the target thermodynamic system. With Bekenstein bound and spherical entropy bound, it is found that the cosmological singularity could be avoided and the cosmological particle production (annihilation) may need to be truncated for some special situations. Our study can be extended to other cosmological models with particle production (annihilation). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03952-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03952) | **An Improved Pair Method to Probe the Dust Extinction Law**  |
|| Yuxi Wang, Jian Gao, Yi Ren, <mark>Jun Li</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| **|
|**Abstract**| Dust extinction law is crucial to recover the intrinsic energy distribution of celestial objects and infer the characteristics of interstellar dust. Based on the traditional pair method, an improved pair method is proposed to model the dust extinguished spectral energy distribution (SED) of an individual star. Instead of the mathematically parameterizing extinction curves, the extinction curves in this work are directly from the silicate-graphite dust model, so that the dust extinction law can be obtained and the dust properties can be analyzed simultaneously. The ATLAS9 stellar model atmosphere is adopted for the intrinsic SEDs in this work, while the silicate-graphite dust model with a dust size distribution of $dn/da \sim a^{-\alpha}{\rm exp}(-a/a_c),~0.005 < a < 5~\mu{\rm m}$ for each component is adopted for the model extinction curves. One typical extinction tracer in the dense region (V410 Anon9) and one in the diffuse region (Cyg OB2 \#12) of the MW are chosen to test the reliability and the practicability of the improved pair method in different stellar environments. The results are consistent with their interstellar environments and are in agreement with the previous observations and studies, which prove that the improved pair method is effective and applicable in different stellar environments. In addition to the reliable extinction results, the derived parameters in the dust model can be used to analyze the dust properties, which cannot be achieved by other methods with the mathematical extinction models. With the improved pair method, the stellar parameters can also be inferred and the extinction law beyond the wavelengths of observed data can be predicted based on the dust model as well. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03420-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03420) | **Parameters of 220 million stars from Gaia BP/RP spectra**  |
|| <mark>Xiangyu Zhang</mark>, <mark>Gregory M. Green</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *27 pages, 22 figures, submitted to MNRAS, Data available at DOI: 10.5281/zenodo.7692680*|
|**Abstract**| We develop, validate and apply a forward model to estimate stellar atmospheric parameters ($T_{\rm eff}$, $\log{g}$ and $\mathrm{[Fe/H]}$), revised distances and extinctions for 220 million stars with XP spectra from $\textit{Gaia}$ DR3. Instead of using $\textit{ab initio}$ stellar models, we develop a data-driven model of $\textit{Gaia}$ XP spectra as a function of the stellar parameters, with a few straightforward built-in physical assumptions. We train our model on stellar atmospheric parameters from the LAMOST survey, which provides broad coverage of different spectral types. We model the $\textit{Gaia}$ XP spectra with all of their covariances, augmented by 2MASS and WISE photometry that greatly reduces degeneracies between stellar parameters, yielding more precise determinations of temperature and dust reddening. Taken together, our approach overcomes a number of important limitations that the astrophysical parameters released in $\textit{Gaia}$ DR3 faced, and exploits the full information content of the data. We provide the resulting catalog of stellar atmospheric parameters, revised parallaxes and extinction estimates, with all their uncertainties. The modeling procedure also produces an estimate of the optical extinction curve at the spectral resolution of the XP spectra ($R \sim 20-100$), which agrees reasonably well with the ${R(V) = 3.1}$ CCM model. Remaining limitations that will be addressed in future work are that the model assumes a universal extinction law, ignores binary stars and does not cover all parts of the Hertzsprung-Russell Diagram ($\textit{e.g.}$, white dwarfs). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \m at position 1</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03464-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03464) | **Momentum Transfer from the DART Mission Kinetic Impact on Asteroid  Dimorphos**  |
|| Andrew F. Cheng, et al. -- incl., <mark>Alex J. Meyer</mark>, <mark>Jian-Yang Li</mark>, <mark>Yun Zhang</mark> |
|*Appeared on*| *2023-03-08*|
|*Comments*| *accepted by Nature*|
|**Abstract**| The NASA Double Asteroid Redirection Test (DART) mission performed a kinetic impact on asteroid Dimorphos, the satellite of the binary asteroid (65803) Didymos, at 23:14 UTC on September 26, 2022 as a planetary defense test. DART was the first hypervelocity impact experiment on an asteroid at size and velocity scales relevant to planetary defense, intended to validate kinetic impact as a means of asteroid deflection. Here we report the first determination of the momentum transferred to an asteroid by kinetic impact. Based on the change in the binary orbit period, we find an instantaneous reduction in Dimorphos's along-track orbital velocity component of 2.70 +/- 0.10 mm/s, indicating enhanced momentum transfer due to recoil from ejecta streams produced by the impact. For a Dimorphos bulk density range of 1,500 to 3,300 kg/m$^3$, we find that the expected value of the momentum enhancement factor, $\beta$, ranges between 2.2 and 4.9, depending on the mass of Dimorphos. If Dimorphos and Didymos are assumed to have equal densities of 2,400 kg/m$^3$, $\beta$= 3.61 +0.19/-0.25 (1 $\sigma$). These $\beta$ values indicate that significantly more momentum was transferred to Dimorphos from the escaping impact ejecta than was incident with DART. Therefore, the DART kinetic impact was highly effective in deflecting the asteroid Dimorphos. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

191  publications in the last 7 days.
	 _build/html/2303.02816.md
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
print(carousel, docs, slides)
print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
  <div class="carousel" 
       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">
    <div class="carousel-cell"> <div id="slide1" class="md_view">Content 1</div> </div>
  </div> "2303.02816.md" "slide1"
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <!-- Remove caching as much as possible -->
    <meta http-equiv="cache-control" content="no-cache" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />
    <!-- flickity bootstrap CSS -->
    <link rel="stylesheet" href="https://unpkg.com/flickity@2/dist/flickity.min.css">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
        integrity="sha384-EVSTQN3/azprG1Anm